In [1]:
import pandas as pd
import glob

In [ ]:
# Define column widths
colspecs = [(2, 34), (35, 36), (36, 37), (37, 42)]

# Read the file
df = pd.read_fwf('../pf.txt', colspecs=colspecs, header=None, names=["address_name", "f", "borough", "sc"], dtype='string')

: 

In [ ]:
# Mappin based on documentation from here: https://data.cityofnewyork.us/City-Government/Street-Name-Dictionary/w4v2-rv6b/about_data
df.loc[df.borough == "1", "borough"] = 'NY'
df.loc[df.borough == "2", "borough"] = 'BX'
df.loc[df.borough == "3", "borough"] = 'K'
df.loc[df.borough == "4", "borough"] = 'Q'
df.loc[df.borough == "5", "borough"] = 'R'

: 

In [ ]:
df[df.sc == '25390'].head()

: 

In [ ]:
base_path = '/d/hpc/projects/FRI/bigdata/students/nk93594/hs/'
files = glob.glob(base_path + "*.csv")

df_list = [pd.read_csv(file) for file in files]
hs = pd.concat(df_list, axis=0, ignore_index=True)

: 

In [ ]:
hs.loc[hs.boro == "M", "boro"] = 'NY'
hs.loc[hs.boro == "X", "boro"] = 'BX'

: 

In [ ]:
# Convert the columns to lowercase
hs['primary_address_line_1'] = hs['primary_address_line_1'].str.lower()
df['address_name'] = df['address_name'].str.lower()

# First, perform an exact match on the 'borough' column
merged = df.merge(hs, left_on='borough', right_on='boro', suffixes=('', '_y'))

: 

In [ ]:
print(list(merged.columns))

: 

In [ ]:
merged['address_name'].head()

: 

In [ ]:
merged['primary_address_line_1'] = merged['primary_address_line_1'].str.replace('^\S+\s*', '', regex=True)

: 

In [ ]:
merged['primary_address_line_1'].head()

: 

In [ ]:
len(hs)

: 

In [ ]:
from fuzzywuzzy import fuzz

# Define a function to apply the fuzzy matching
def fuzzy_match(row, threshold=90):  # increase the threshold for a near-perfect match
    return fuzz.token_sort_ratio(row['primary_address_line_1'], row['address_name']) > threshold

# Apply the fuzzy matching
mask = merged.apply(fuzzy_match, axis=1)

# Keep only the rows where the two columns match
len(merged[mask])

: 

In [ ]:
temp = merged[mask]

temp = temp[temp.address_name != temp.primary_address_line_1]
temp[['address_name', 'primary_address_line_1', 'borough', 'boro']].head(50)

: 